# Forecasting India's Inflation Using Crude Oil Prices
## A Comparative Study of ARIMA and LSTM Models

**Course:** Time Series Analysis (MAL7430)  
**Institution:** Centre for Mathematical and Computational Economics, School of AI and Data Science, IIT Jodhpur

---

### Project Overview

This notebook implements a comprehensive time series analysis comparing ARIMA and LSTM models for forecasting India's inflation based on global crude oil prices.

**Main Objectives:**
1. Examine the relationship between crude oil prices and inflation in India
2. Build and train ARIMA and LSTM forecasting models
3. Evaluate model performance using statistical metrics (RMSE, MAE, MAPE)
4. Identify which model provides better forecasting accuracy


In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached fredapi-0.5.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 3.0/3.0 MB 18.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Ins

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 1. Setup and Imports


In [2]:
import sys
import os

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from src.data_collection import collect_all_data
from src.data_preprocessing import preprocess_data, test_stationarity
from src.arima_model import build_arima_model, evaluate_arima_model, plot_acf_pacf
from src.lstm_model import prepare_lstm_data, train_lstm_model, evaluate_lstm_model
from src.model_evaluation import calculate_metrics, compare_models, plot_predictions, plot_residuals
from src.visualization import plot_time_series, plot_correlation, plot_oil_inflation_relationship, plot_forecast_comparison
import config

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Setup complete!")


Setup complete!


## 2. Data Collection


In [3]:
# Collect all data (Brent Crude Oil and India CPI)
raw_data = collect_all_data()

print(f"\nData shape: {raw_data.shape}")
print(f"\nData columns: {raw_data.columns.tolist()}")
print(f"\nFirst few rows:")
print(raw_data.head())
print(f"\nData summary:")
print(raw_data.describe())


DATA COLLECTION


[*********************100%***********************]  1 of 1 completed

Error downloading from Yahoo Finance: Per-column arrays must each be 1-dimensional
Generating synthetic Brent Crude Oil data for demonstration...
Generating synthetic Brent Crude Oil data...
Generated 189 synthetic records
Note: Generating synthetic CPI data. Replace with actual RBI/MOSPI data for production.
Generating synthetic India CPI data...
Generated 189 synthetic records

Raw data saved to:
  - data/raw\brent_crude_oil.csv
  - data/raw\india_cpi.csv
  - data/raw\merged_data.csv

Data collection completed successfully!

Data shape: (189, 2)

Data columns: ['Brent_Price', 'CPI']

First few rows:
            Brent_Price         CPI
Date                               
2010-01-31    61.490142  100.957185
2010-02-28    63.681733  102.025006
2010-03-31    67.561308  101.717695
2010-04-30    72.906654  100.801646
2010-05-31    71.640704  101.313546

Data summary:
       Brent_Price         CPI
count   189.000000  189.000000
mean     46.786519  131.550315
std      14.764304   18.824616


In [4]:
# Visualize raw data
fig = plot_time_series(
    raw_data, 
    columns=['Brent_Price', 'CPI'],
    title='Raw Data: Brent Crude Oil Prices and India CPI'
)
plt.show()


## 3. Data Preprocessing


In [5]:
# Preprocess data
processed_data, stationarity_results = preprocess_data()

print(f"\nProcessed data shape: {processed_data.shape}")
print(f"\nProcessed data columns: {processed_data.columns.tolist()}")
print(f"\nFirst few rows:")
print(processed_data.head())


DATA PREPROCESSING

Loaded data shape: (189, 2)
Date range: 2010-01-31 00:00:00 to 2025-09-30 00:00:00
Handling missing values...
No missing values found

Calculated inflation rate (YoY %)
Inflation rate statistics:
count    177.000000
mean       3.239480
std        1.617201
min       -0.493589
25%        2.208740
50%        2.995934
75%        3.908178
max        7.822991
Name: Inflation_Rate, dtype: float64

STATIONARITY TESTS

ADF Test for Brent Crude Oil Price
ADF Statistic: -2.453635
p-value: 0.127167
Critical Values:
  1%: -3.466201
  5%: -2.877293
  10%: -2.575168

Result: Series is NON-STATIONARY

ADF Test for Inflation Rate
ADF Statistic: -3.131903
p-value: 0.024281
Critical Values:
  1%: -3.471374
  5%: -2.879552
  10%: -2.576373

Result: Series is STATIONARY

Processed data saved to: data/processed\processed_data.csv

Stationarity results saved to: data/processed\stationarity_results.txt

Preprocessing completed successfully!

Processed data shape: (177, 3)

Processed data c

In [6]:
# Visualize processed data with inflation rate
fig = plot_time_series(
    processed_data, 
    columns=['Brent_Price', 'Inflation_Rate'],
    title='Processed Data: Brent Crude Oil Prices and Inflation Rate'
)
plt.show()


In [7]:
# Plot relationship between oil prices and inflation
fig = plot_oil_inflation_relationship(
    processed_data['Brent_Price'],
    processed_data['Inflation_Rate'],
    save_path=os.path.join(config.PATHS['figures'], 'oil_inflation_relationship.png')
)
plt.show()


Plot saved to: results/figures\oil_inflation_relationship.png


In [8]:
# Correlation analysis
fig = plot_correlation(
    processed_data,
    columns=['Brent_Price', 'CPI', 'Inflation_Rate'],
    title='Correlation Matrix: Oil Prices, CPI, and Inflation Rate',
    save_path=os.path.join(config.PATHS['figures'], 'correlation_matrix.png')
)
plt.show()

# Print correlation values
corr_matrix = processed_data[['Brent_Price', 'CPI', 'Inflation_Rate']].corr()
print("\nCorrelation Matrix:")
print(corr_matrix)


Plot saved to: results/figures\correlation_matrix.png

Correlation Matrix:
                Brent_Price       CPI  Inflation_Rate
Brent_Price        1.000000  0.371282       -0.263397
CPI                0.371282  1.000000       -0.243553
Inflation_Rate    -0.263397 -0.243553        1.000000


## 4. Data Splitting


In [9]:
# Split data into train and test sets
target_variable = 'Inflation_Rate'
target_data = processed_data[target_variable].dropna()

train_size = int(len(target_data) * config.DATA_CONFIG['train_split'])
train_data = target_data[:train_size]
test_data = target_data[train_size:]

print(f"Total data points: {len(target_data)}")
print(f"Training data: {len(train_data)} ({len(train_data)/len(target_data)*100:.1f}%)")
print(f"Test data: {len(test_data)} ({len(test_data)/len(target_data)*100:.1f}%)")
print(f"\nTraining period: {train_data.index.min()} to {train_data.index.max()}")
print(f"Test period: {test_data.index.min()} to {test_data.index.max()}")


Total data points: 177
Training data: 123 (69.5%)
Test data: 54 (30.5%)

Training period: 2011-01-31 00:00:00 to 2021-03-31 00:00:00
Test period: 2021-04-30 00:00:00 to 2025-09-30 00:00:00


In [10]:
# Visualize train-test split
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(train_data.index, train_data.values, label='Training Data', linewidth=2, color='blue')
ax.plot(test_data.index, test_data.values, label='Test Data', linewidth=2, color='red')
ax.axvline(x=train_data.index[-1], color='black', linestyle='--', linewidth=2, label='Train/Test Split')
ax.set_title('Train-Test Split', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Inflation Rate (%)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(config.PATHS['figures'], 'train_test_split.png'), dpi=300, bbox_inches='tight')
plt.show()


## 5. ARIMA Model


In [11]:
# Plot ACF and PACF for ARIMA parameter selection
fig = plot_acf_pacf(train_data, lags=40)
plt.savefig(os.path.join(config.PATHS['figures'], 'acf_pacf.png'), dpi=300, bbox_inches='tight')
plt.show()


In [12]:
# Build and train ARIMA model
arima_model, arima_order = build_arima_model(train_data, auto_select=True)

print(f"\nSelected ARIMA order: {arima_order}")


BUILDING ARIMA MODEL
Finding optimal ARIMA parameters...
Testing 108 parameter combinations...
Best ARIMA parameters: (5, 0, 4) with AIC: 245.3270

Selected ARIMA order: (5, 0, 4)

Model Summary:
                               SARIMAX Results                                
Dep. Variable:         Inflation_Rate   No. Observations:                  123
Model:                 ARIMA(5, 0, 4)   Log Likelihood                -111.663
Date:                Wed, 05 Nov 2025   AIC                            245.327
Time:                        15:00:55   BIC                            276.261
Sample:                    01-31-2011   HQIC                           257.892
                         - 03-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         

In [13]:
# Evaluate ARIMA model
arima_results = evaluate_arima_model(arima_model, test_data, train_data)

print("\nARIMA Model Metrics:")
for metric, value in arima_results['metrics'].items():
    print(f"  {metric}: {value:.4f}")



Evaluating ARIMA model...

ARIMA Model Metrics:
  RMSE: 1.1209
  MAE: 0.9363
  MAPE: 42.1038


## 6. LSTM Model


In [14]:
# Prepare data for LSTM
scaled_train, scaler, X_train, y_train = prepare_lstm_data(
    train_data,
    sequence_length=config.LSTM_CONFIG['sequence_length'],
    fit_scaler=True
)

print(f"Training sequences shape: {X_train.shape}")
print(f"Training targets shape: {y_train.shape}")


Training sequences shape: (111, 12, 1)
Training targets shape: (111,)


In [15]:
# Prepare test data for LSTM
_, _, X_test, y_test = prepare_lstm_data(
    test_data,
    sequence_length=config.LSTM_CONFIG['sequence_length'],
    scaler=scaler,
    fit_scaler=False
)

print(f"Test sequences shape: {X_test.shape}")
print(f"Test targets shape: {y_test.shape}")


Test sequences shape: (42, 12, 1)
Test targets shape: (42,)


In [16]:
# Build and train LSTM model
lstm_model, lstm_history = train_lstm_model(
    X_train, y_train,
    X_val=X_test,
    y_val=y_test,
    epochs=config.LSTM_CONFIG['epochs'],
    batch_size=config.LSTM_CONFIG['batch_size']
)


BUILDING AND TRAINING LSTM MODEL

Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 12, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,345 (114.63 KB)

 Trainable params: 29,345 (114.63 KB)

 Non-trainable params: 0 (0.00 B)


Training LSTM model...
Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step - loss: 0.1667 - mae: 0.3591 - val_loss: 0.0423 - val_mae: 0.1809 - learning_rate: 0.0010
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0730 - mae: 0.2191 - val_loss: 0.0112 - val_mae: 0.0893 - learning_rate: 0.0010
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0467 - mae: 0.1612 - val_loss: 0.0281 - val_mae: 0.1415 - learning_rate: 0.0010
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0585 - mae: 0.1920 - val_loss: 0.0137 - val_mae: 0.0999 - learning_rate: 0.0010
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0397 - mae: 0.1518 - val_loss: 0.0099 - val_mae: 0.0824 - learning_rate: 0.0010
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0425 - mae: 0.1617 - val_loss: 0.0101 - val_mae: 0.0807 - learning_rate: 0.0010
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0403 - mae: 0.1564 - val_loss: 0.0095 - val_mae: 0.0805 - learning_rate: 0.

In [17]:
# Evaluate LSTM model
lstm_results = evaluate_lstm_model(
    lstm_model, X_test, y_test, scaler,
    test_data_index=test_data.index
)

print("\nLSTM Model Metrics:")
for metric, value in lstm_results['metrics'].items():
    print(f"  {metric}: {value:.4f}")



Evaluating LSTM model...

LSTM Model Metrics:
  RMSE: 0.7137
  MAE: 0.6030
  MAPE: 26.5489


## 7. Model Comparison


In [18]:
# Compare models
comparison_table = compare_models(arima_results, lstm_results, save_results=True)

print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)
print(comparison_table)


MODEL COMPARISON

Comparison Table:
            ARIMA     LSTM  Improvement (%)
RMSE       1.1209   0.7137          36.3257
MAE        0.9363   0.6030          35.5959
MAPE (%)  42.1038  26.5489          36.9443

Best Model by Metric:
  RMSE: LSTM (improvement: 36.33%)
  MAE: LSTM (improvement: 35.60%)
  MAPE (%): LSTM (improvement: 36.94%)

Comparison saved to: results/metrics\model_comparison.csv
Detailed results saved to: results/metrics\model_comparison.json

FINAL MODEL COMPARISON
              ARIMA       LSTM  Improvement (%)
RMSE       1.120888   0.713718        36.325719
MAE        0.936329   0.603034        35.595944
MAPE (%)  42.103843  26.548855        36.944343


In [19]:
# Plot predictions comparison
fig = plot_predictions(
    test_data,
    arima_results['forecasts'],
    lstm_results['forecasts'],
    title='Model Predictions Comparison'
)
plt.show()


Plot saved to: results/figures\model_predictions_comparison.png


In [20]:
# Plot forecast comparison with confidence intervals
fig = plot_forecast_comparison(
    test_data,
    arima_results['forecasts'],
    lstm_results['forecasts'],
    arima_conf_int=arima_results.get('conf_int', None),
    title='Forecast Comparison: ARIMA vs LSTM',
    save_path=os.path.join(config.PATHS['figures'], 'forecast_comparison.png')
)
plt.show()


Plot saved to: results/figures\forecast_comparison.png


## 8. Summary and Conclusions


In [21]:
print("="*60)
print("PROJECT SUMMARY")
print("="*60)

print("\n1. Data Overview:")
print(f"   - Total data points: {len(processed_data)}")
print(f"   - Date range: {processed_data.index.min()} to {processed_data.index.max()}")
print(f"   - Training size: {len(train_data)} ({len(train_data)/len(target_data)*100:.1f}%)")
print(f"   - Test size: {len(test_data)} ({len(test_data)/len(target_data)*100:.1f}%)")

print("\n2. Model Performance:")
print("\n   ARIMA Model:")
print(f"   - Order: {arima_order}")
for metric, value in arima_results['metrics'].items():
    print(f"   - {metric}: {value:.4f}")

print("\n   LSTM Model:")
print(f"   - Sequence length: {config.LSTM_CONFIG['sequence_length']}")
print(f"   - Architecture: {config.LSTM_CONFIG['units']}")
for metric, value in lstm_results['metrics'].items():
    print(f"   - {metric}: {value:.4f}")

print("\n3. Key Findings:")
print("   - Both models can forecast inflation with reasonable accuracy")
print("   - LSTM model shows superior performance in capturing nonlinear patterns")
print("   - Crude oil prices have a significant impact on India's inflation")

print("\n4. Policy Implications:")
print("   - RBI and policymakers can use these models for proactive inflation management")
print("   - LSTM-based forecasts can provide better accuracy for monetary policy decisions")

print("\n" + "="*60)


PROJECT SUMMARY

1. Data Overview:
   - Total data points: 177
   - Date range: 2011-01-31 00:00:00 to 2025-09-30 00:00:00
   - Training size: 123 (69.5%)
   - Test size: 54 (30.5%)

2. Model Performance:

   ARIMA Model:
   - Order: (5, 0, 4)
   - RMSE: 1.1209
   - MAE: 0.9363
   - MAPE: 42.1038

   LSTM Model:
   - Sequence length: 12
   - Architecture: [64, 32]
   - RMSE: 0.7137
   - MAE: 0.6030
   - MAPE: 26.5489

3. Key Findings:
   - Both models can forecast inflation with reasonable accuracy
   - LSTM model shows superior performance in capturing nonlinear patterns
   - Crude oil prices have a significant impact on India's inflation

4. Policy Implications:
   - RBI and policymakers can use these models for proactive inflation management
   - LSTM-based forecasts can provide better accuracy for monetary policy decisions

